In [1]:
import json
import pandas as pd
from glob import glob
import os

In [2]:
with open('../download_detailed.json') as o:
	downloads = json.loads(o.read())

In [3]:
sab_dict = {
	"LINCS": ["LINCS"],
	"4DN": ["4DN"],
	"ERCC": ["ERCCRBP", "ERCCREG"],
	"GlyGen": ["PROTEOFORM", "GLYCANS"],
	"GTEx": ["GTEXEXP", "GTEXEQTL"],
	"HuBMAP": ["AZ", "HMAZ"],
	"IDG": ["IDGP", "IDGD"],
	"KF": ["KF"],
	"MoTrPAC": ["MOTRPAC"],
	"MW": ["MW"],
	"SPARC": ["NPO"],
	"CLINVAR": ["CLINVAR"],
	"HGNCHPO": ["HGNCHPO"],
	"MSIGDB": ["MSIGDB"],
	"HGNCUNIPROT": ["HGNCUNIPROT"],
	"HGNCENZ": ["HGNCENZ"]
}


In [8]:
desc = {}
for i in downloads:
	desc[i["source"]] = i["description"]

In [9]:
desc

{'GTEX': 'Expression and eQTL data from GTEx',
 'HGNCHPO': 'HGNC gene node mapping to Human Phenotype Ontology',
 'MoTrPAC': 'Differential gene expression data from endurance training exercise of young adult rats',
 'KF': 'Genotypic and phenotypic data from the Pediatric Cardiac Genetics Consortium cohort in Kids First',
 '4DN': 'Chromatin loops called from Hi-C experiments performed in select cell lines',
 'LINCS': 'Gene expression data from drug perturbation, as well as drug-drug similarity',
 'Metabolomics Workbench': 'Metabolite relationships with gene, disease, and cell',
 'CLINVAR': 'Assertions between human genes and phenotypes',
 'GLYGEN': 'Associations from multiple glycomics database',
 'ERCC': 'eCLIP-seq and CHIP-seq associations from ERCC',
 'MSIGDB': 'Five subsets of MSigDB v7.4 datasets',
 'IDG': 'Relationships between compounds, diseases, and proteins'}

In [ ]:
dcc_descriptions = {
	'LINCS': 'Gene expression data from drug perturbation, as well as drug-drug similarity',
	'4DN': 'Chromatin loops called from Hi-C experiments performed in select cell lines',
	'ERCC': 'eCLIP-seq and CHIP-seq associations from ERCC',
	'GlyGen': 'Associations from multiple glycomics database',
	'GTEx': 'Expression and eQTL data from GTEx',
	"HuBMAP": "Tissue, cell-type and gene specific markers from single-cell data",
	'IDG': 'Relationships between compounds, diseases, and proteins',
	'KF': 'Genotypic and phenotypic data from the Pediatric Cardiac Genetics Consortium cohort in Kids First',
	'MoTrPAC': 'Differential gene expression data from endurance training exercise of young adult rats',
	'MW': 'Metabolite relationships with gene, disease, and cell',
	"SPARC": "The SPARC Knowledge base of the Automatic Nervous System (SCKAN)",
	'CLINVAR': 'Assertions between human genes and phenotypes',
	'HGNCHPO': 'HGNC gene node mapping to Human Phenotype Ontology',
	'MSIGDB': 'Five subsets of MSigDB v7.4 datasets',
	"HGNCUNIPROT": "Gene-Protein relationships",
	"HGNCENZ": "Enzyme Genes",
}

In [4]:
def get_file_size(filename):
    size = float(os.path.getsize(filename))
    if size > 1000000000:
        return "%.2fGB"%(size/1000000000)
    if size > 1000000:
        return "%.2fMB"%(size/1000000)
    if size > 1000:
        return "%.2fKB"%(size/1000)
    else: return "%d bytes"%int(size)

In [7]:
downloads[0]

{'source': 'GTEX',
 'description': 'Expression and eQTL data from GTEx',
 'terms': 2862091,
 'edges': 36369763,
 'url': 'https://s3.amazonaws.com/maayan-kg/dd-kg/042624/assertions/GTEX.edges.zip',
 'size': '4.35GB',
 'updated': '10-26-2023'}

In [5]:
downloads = []
for filename in glob('out/compressed/*.zip'):
	dcc = filename.replace("out/compressed/", "").replace(".zip", "")
	meta = {
		"source": dcc,
		"url": "https://s3.amazonaws.com/maayan-kg/dd-kg/042624/%s.zip"%dcc,
		"size": get_file_size(filename)
	}
	nodes = set()
	edge_count = 0
	for sab in sab_dict[dcc]:
		df = pd.read_csv("out/sab/%s.edges.csv"%sab)
		edge_count = edge_count + len(df.index)
		nodes = nodes.union(df.source).union(df.target)
	meta["edges"] = edge_count
	meta["nodes"] = len(nodes)

In [6]:
filesize

'51.62MB'